In [1]:
# Dependencies
import numpy as np
import pandas as pd
import requests
import json
# ------- STUFF WE MIGHT NEED LATER ----------
# import matplotlib.pyplot as plt
# import gmaps
import urllib
import time
# import seaborn as sbn
# from scipy import stats


# # Gmaps API Keys
# from config import (gkey)

# # Configure gmaps
# gmaps.configure(api_key=gkey)

# Creating an array of ranges to iterate through all the API calls
arr = np.arange(0, 178000, 2000)
arr

array([     0,   2000,   4000,   6000,   8000,  10000,  12000,  14000,
        16000,  18000,  20000,  22000,  24000,  26000,  28000,  30000,
        32000,  34000,  36000,  38000,  40000,  42000,  44000,  46000,
        48000,  50000,  52000,  54000,  56000,  58000,  60000,  62000,
        64000,  66000,  68000,  70000,  72000,  74000,  76000,  78000,
        80000,  82000,  84000,  86000,  88000,  90000,  92000,  94000,
        96000,  98000, 100000, 102000, 104000, 106000, 108000, 110000,
       112000, 114000, 116000, 118000, 120000, 122000, 124000, 126000,
       128000, 130000, 132000, 134000, 136000, 138000, 140000, 142000,
       144000, 146000, 148000, 150000, 152000, 154000, 156000, 158000,
       160000, 162000, 164000, 166000, 168000, 170000, 172000, 174000,
       176000])

In [2]:
#This API request somehow limits the number of records returned to 2000. But we can download all of them from the site
# as a json file. However, if we want we can iterate through the 178000 records 89 times and get them all :)

base_url = "https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/Fire_Incidents_Public/FeatureServer/0/query?"

field_list = "OBJECTID,incident_number,incident_type,incident_type_description,arrive_date_time,dispatch_date_time,exposure,platoon,station,address,address2,apt_room,GlobalID,cleared_date_time"
parameters = f"where=1%3D1&outFields={field_list}&outSR=4326&f=json"

set_length = 1

# resultsOffset is the record to start with, resultRecordCount is the number of records to return, max=2000
#For iterating through API calls to get all the data. We know there are 177,738 Rows, so we need to run this 89 times.

records = f'&resultOffset={set_length}&resultRecordCount=2000'

# fire_data = requests.get(base_url + parameters + records).json()


In [5]:
# Looks like we can iterate through the calls to get all the records.
# Here is a function that iterates through a list and returns all the JSON data to a hard file

def arcgis_api(sets):
    fire_data = []
    # Work with the log file to record the API calls/errors
    timestr = time.strftime('%Y%m%d-%H%M%S')
    log = 'log_' + timestr + '.txt'
    print(f'Logging API calls in {log}')

    with open(log, 'a+') as f:

        for n in sets:
            status = (f'Getting data for records {n}: {n+2000}... ')
            print(status)
            #Perform the API call on ARCGIS
            try:
                url = base_url + parameters + f'&resultOffset={n}&resultRecordCount=2000'
                records = requests.get(url).json()
                f.write(url)
                fire_data.append(records)
            #Log the result
                msg = 'Success!\n'
                f.write(msg)
                print(msg)
                time.sleep(5)
            except requests.exceptions.ContentDecodingError as e:
                msg = f'There was an error with the set starting with record {n}\n'
                f.write(msg)
                print(msg)

    with open('fire' + timestr + '.txt', 'a+') as file:
        print(f'Dumping json data to {file}')
        json.dump(fire_data, file)



In [6]:
# Calling the function with the array created above
# that will do the 89 API calls and save all the data to the fire_timestamp_txt file
# There will also be a logfile of the transactions
arcgis_api(arr)


Logging API calls in log_20190111-132414.txt
Getting data for records 0: 2000... 
Success!

Getting data for records 2000: 4000... 
Success!

Getting data for records 4000: 6000... 
Success!

Getting data for records 6000: 8000... 
Success!

Getting data for records 8000: 10000... 
Success!

Getting data for records 10000: 12000... 
Success!

Getting data for records 12000: 14000... 
Success!

Getting data for records 14000: 16000... 
Success!

Getting data for records 16000: 18000... 
Success!

Getting data for records 18000: 20000... 
Success!

Getting data for records 20000: 22000... 
Success!

Getting data for records 22000: 24000... 
Success!

Getting data for records 24000: 26000... 
Success!

Getting data for records 26000: 28000... 
Success!

Getting data for records 28000: 30000... 
Success!

Getting data for records 30000: 32000... 
Success!

Getting data for records 32000: 34000... 
Success!

Getting data for records 34000: 36000... 
Success!

Getting data for records 36000:

In [51]:
firefile = '122000.json'
with open(firefile) as jsonfile:
    fire_data = json.load(jsonfile)

In [52]:
#Once we have all the json information we can parse them out as such
incidents = []
lat = []
lon = []
# I'm stuck here: The call data for each
for i in fire_data['features']:
    incidents.append(i['attributes'])
incidents[0]

{'OBJECTID': 594765,
 'incident_number': '15-0022476',
 'incident_type': 611,
 'incident_type_description': 'Dispatched & cancelled en route',
 'arrive_date_time': None,
 'dispatch_date_time': 1437760020000,
 'exposure': 0,
 'platoon': 'B',
 'station': 2,
 'address': '911 ILEAGNES RD RALEIGH, NC 27601',
 'address2': 'PEACH RD COMM CTR                       ',
 'apt_room': '               ',
 'GlobalID': '273f1785-39de-4855-98a6-894f59de6fdd',
 'cleared_date_time': 1437760200000}

In [59]:
# For the life of me I can't see where I am going wrong here....

for i in fire_data['features']:
    lon.append(i["geometry"])
#     lat.append(i['geometry']['y'])
fire_data['features'][0]['geometry']

KeyError: 'geometry'

In [62]:
fire_data['features'][0]['geometry']['x']

-78.65512932825119

In [13]:
incidents = []
lon = []
lat = []
for item in fire_data:
    incidents.append(item['features'])
len(incidents)    

88

In [36]:
incidents[0]

[{'attributes': {'OBJECTID': 474765,
   'incident_number': '07-0031665',
   'incident_type': None,
   'incident_type_description': 'NULL',
   'arrive_date_time': 1195125420000,
   'dispatch_date_time': 1195125000000,
   'exposure': 0,
   'platoon': ' ',
   'station': None,
   'address': '6647 LAKE HILL DR RALEIGH, NC 27601',
   'address2': '',
   'apt_room': '0',
   'GlobalID': '5c2e9c89-78d7-4348-bd95-23a69f4039aa',
   'cleared_date_time': 1195125900000},
  'geometry': {'x': -78.62660452385826, 'y': 35.870212857675455}},
 {'attributes': {'OBJECTID': 474766,
   'incident_number': '08-0017918',
   'incident_type': None,
   'incident_type_description': 'NULL',
   'arrive_date_time': 1214720400000,
   'dispatch_date_time': 1214720220000,
   'exposure': 0,
   'platoon': ' ',
   'station': None,
   'address': '539 METHOD RD RALEIGH, NC 27606',
   'address2': '',
   'apt_room': '               ',
   'GlobalID': 'b6bdde67-94b6-409d-a17d-55498242caf1',
   'cleared_date_time': 1214721720000},
 